In [1]:
import pandas as pd
import pickle
import numpy as np
import requests
import plotly.express as px
import scipy.optimize as opt

In [2]:
survey_path = "../../../results/road/survey.parquet"
calibration_path = "../../../results/road/freeflow/calibration_cache_api.pickle"

routing_endpoint = "http://localhost:8054/router/road"
departure_time = 4 * 3600
maximum_batch_size = 400

output_path = "../../../results/road/congestion_factors.parquet"

In [3]:
# Load survey data
df_survey = pd.read_parquet(survey_path)

In [4]:
# Load freeflow routing parameters
with open(calibration_path, "rb") as f:
    history = pickle.load(f)

objective = np.inf
best = None

for item in history:
    if item["objective"] < objective:
        best = item
        # break # use first

settings = best["settings"]
settings

{'major_factor': 1.0000000000964675,
 'intermediate_factor': 1.000000000110372,
 'minor_factor': 1.0000000000071958,
 'major_crossing_penalty_s': 4.20563605332865,
 'minor_crossing_penalty_s': 1.1414043230895694e-09}

In [5]:
# Prepare requests
df_survey["request_index"] = np.arange(len(df_survey))

# Convert to requests
request_list = []

for index, row in df_survey.iterrows():
    request_list.append({
        "request_index": int(row["request_index"]),
        "origin_x": row["origin_x"],
        "origin_y": row["origin_y"],
        "destination_x": row["destination_x"],
        "destination_y": row["destination_y"],
        "departure_time_s": departure_time
    })

In [6]:
# Prepare querying
def query_requests(request_list, settings):
    df_response = []
    batch_index = 0

    while batch_index * maximum_batch_size < len(request_list):
        batch = request_list[batch_index * maximum_batch_size : (batch_index + 1) * maximum_batch_size]

        response = requests.post(routing_endpoint, json = {
            "batch": batch,
            "freespeed": settings
        })

        df_response.append(pd.DataFrame.from_records(response.json()))
        batch_index += 1

    return pd.concat(df_response)

In [7]:
# Test connection
assert len(query_requests(request_list[:5], settings)) == 5

In [8]:
# Obtain response
df_response = query_requests(request_list, settings)

In [9]:
# Inject into surey data (and only compare pure driving time)
df_survey["freeflow_travel_time_s"] = df_response["in_vehicle_time_min"].values * 60.0
df_survey["survey_travel_time_s"] -= df_response["access_time_min"].values * 60.0
df_survey["survey_travel_time_s"] -= df_response["egress_time_min"].values * 60.0

# Calculate hour
df_survey["hour"] = df_survey["departure_time"] // 3600
df_survey.loc[df_survey["hour"] > 23, "hour"] -= 24
df_survey["hour"] = df_survey["hour"].astype(int)

In [10]:
df_survey = df_survey[df_survey["freeflow_travel_time_s"] > 300]
df_survey = df_survey[df_survey["survey_travel_time_s"] > 300]
df_survey = df_survey[df_survey["survey_travel_time_s"] >= df_survey["freeflow_travel_time_s"]]

In [11]:
def objective(x):
    weight = df_survey["weight"]
    reference = df_survey["survey_travel_time_s"].values
    scaled = df_survey["freeflow_travel_time_s"].values * x[df_survey["hour"].values]
    return 1e-3 * np.sum(np.abs(reference - scaled) * weight) / np.sum(weight)

result = opt.minimize(objective, np.ones(24), method = "Powell")
assert result.success

In [12]:
df_result = pd.DataFrame({
    "hour": np.arange(24),
    "congestion_factor": result.x
})

In [13]:
px.line(df_result, x = "hour", y = "congestion_factor")

In [14]:
df_result.to_parquet(output_path)